Generate STAC Collections for NOAA Climate Data Record

In [26]:
import sys
from datetime import datetime
import argparse
import json
import fsspec
import numpy
import pandas as pd
import packaging
import xarray as xr
from pathlib import Path
import earthpy as et

import xstac
import pystac
import shapely.geometry

In [2]:
BBOX = {
    "north": [31.1, -180, 89.84, 180],
    "south": [-89.84, -180, -39.36, 180],
}
    

In [3]:
DESC = {
    "daily":"",
    "monthly":"",
    "aggregate":"",
}

In [4]:
CITATION_URLS = {
    "daily": "https://doi.org/10.7265/efmz-2t65",
    "monthly": "https://doi.org/10.7265/efmz-2t65",
    "annual": "https://doi.org/10.7265/efmz-2t65",
}

CITATION = {("Meier, W. N., F. Fetterer, A. K. Windnagel, and S. Stewart. 2021. NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration, Version 4."
             "[Indicate subset used]. Boulder, Colorado, USA. NSIDC: National Snow and Ice Data Center https://doi.org/10.7265/efmz-2t65.")}

In [5]:
FREQUENCIES = ["daily", "monthly"]
# TODO: Add aggregate later

In [6]:
REGIONS = ["north", "south"]

In [7]:
# Unsure if this is correct
NETCDF_MEDIA_TYPE = pystac.MediaType.HDF

In [8]:
# Generate list of years
start_year = (datetime(1978, 1, 1).year)
current_year = (datetime.now()).year
years = list(range(start_year, 1 + current_year))
str_years = [str(year) for year in years]
print(str_years)

['1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [9]:
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [10]:
doys = list(range(1, 366))
str_doys = [str(doy) for doy in doys]
doys1 = [ "00"+doy for doy in str_doys if len(doy)==1]
doys2 = [ "0"+doy for doy in str_doys if len(doy)==2]
doys3 = [doy for doy in str_doys  if len(doy)==3]
full_doys = doys1 + doys2 + doys3
print(full_doys)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143'

In [11]:
def parse_args(args=None):
    parser = argparse.ArgumentParser(usage=__doc__)
    parser.add_argument(
        "--region", type=str, choices=["both", "north", "south"], default="both"
    )
    parser.add_argument(
        "--frequency",
        type=str,
        choices=["all", "daily", "monthly"], #aggregate
        default="all",
    )
    parser.add_argument(
        "--year",
        type=str,
        choices=str_years,
        default='1980',
    )
    return parser.parse_args(args)

In [12]:
#url = ("https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/monthly/seaice_conc_monthly_sh_197902_n07_v04r00.nc")
# file = "~/Downloads/seaice_conc_monthly_sh_197811_n07_v04r00.nc"
# data = et.data.get_data(url=url)
# ds = xr.open_dataset(data)
# print(ds)

In [23]:
def generate(frequency, region, year):

    collection_template = {
        "id": f"seaiceClimateDataRecordV4-{frequency}-{region}",
        "stac_extensions": [
            "https://stac-extensions.github.io/scientific/v1.0.0/schema.json"
        ],
        "description": "{{ collection.description }}",
        "type": "Collection",
        "title": f"NOAA/NSIDC passive microwave sea ice concentration climate data record {frequency.title()} {region}",
        "license": "proprietary",
        "keywords": [
            "EARTH SCIENCE",
            "CRYOSPHERE",
            "SEA ICE",
            "SEA ICE CONCENTRATION",
            "Polar",
        ],
        "stac_version": "1.0.0",
        "links": [
            {
                "rel": "license",
                "title": "EOSDIS Data Use Policy",
                "href": "https://science.nasa.gov/earth-science/earth-science-data/data-information-policy",
            },
            {"rel": "cite-as", "href": CITATION},
        ],
        "extent": {
            "spatial": {"bbox": [BBOX[region]]},
            },
        "providers": [
            {
                "name": "NOAA at the National Snow and Ice Data Center",
                "roles": ["licensor", "host", "processor"],
                "url": "https://nsidc.org/data",
            },
        ],
        "assets": {
            "netcdf-https": {
                "href": f"https://noaadata.apps.nsidc.org/NOAA/G02202_V4/{region}/{frequency}.nc",
                "type": "application/x-hdf",
                "roles": ["data", "netcdf4", "https"]},
               #  "xarray:open_kwargs": {"consolidated": True},
        },
        "sci:doi": CITATION_URLS,
        "sci:citation": CITATION,
    }

    
    if region == "north":
        region_sn = "nh"
    else:
        region_sn = "sh"

    if frequency == "daily":
        # for doy in doys: 
        filepath_url = f"https://noaadata.apps.nsidc.org/NOAA/G02202_V4/{region}/daily/{year}" # /seaice_conc_daily_{region_sn}_{year}{doy}_n07_v04r00.nc"
        fs = fsspec.filesystem('https') #, host=host)
        url = filepath_url
        files = [f for f in fs.find(url) if f.endswith('.nc')]
        with fs.open(files[0]) as f:
            ds = xr.open_dataset(f)
    elif frequency == "monthly":
        # for month in months:
        filepath_url = f"https://noaadata.apps.nsidc.org/NOAA/G02202_V4/{region}/monthly" # /seaice_conc_monthly_{region_sn}_{year}{month}_n07_v04r00.nc" 
        fs = fsspec.filesystem('https') #, host=host)
        url = filepath_url
        files = [f for f in fs.find(url) if f.endswith('.nc')]
        
        with fs.open(files[0]) as f:
            ds = xr.open_dataset(f)



    collection = xstac.xarray_to_stac(
        ds,
        collection_template,
        temporal_dimension="time",
        x_dimension="x",
        y_dimension="y",
    )
    collection.remove_links(pystac.RelType.SELF)
    collection.remove_links(pystac.RelType.ROOT)
    collection_result = collection.to_dict(include_self_link=False)
    
    item_template = {
        "id": f"cdr-{frequency}-{region}",
        "type": "Feature",
        "links": [],
        "bbox": BBOX[region],
        "geometry": shapely.geometry.mapping(shapely.geometry.box(*BBOX[region])),
        "stac_version": "1.0.0",
        "properties": {"start_datetime": None, "end_datetime": None},
        "assets": {
            "hdf-https": {
                "href": f"https://noaadata.apps.nsidc.org/NOAA/G02202_V4/{region}/{frequency}.nc",
                "type": "application/x-hdf",
                "title": f"{frequency.title()} {region} CDR HTTPS NetCDF root",
                "description": f"HTTPS URI of the {frequency} {region} Climate Data Record.",  # noqa: E501
                "roles": ["data", "hdf", "https"],
                "xarray:open_kwargs": {"consolidated": True},
            },
        },
    }
    item = xstac.xarray_to_stac(
        ds, item_template, temporal_dimension="time", x_dimension="x", y_dimension="y"
    )
    
    item_result = item.to_dict(include_self_link=False)
    return collection_result, item_result



In [17]:
def main(args=None):
    args = parse_args(args)
    region = args.region
    frequency = args.frequency
    year = args.year

    if region == "all":
        regions = list(REGIONS)
    else: 
        regions = [region]

    if frequency == "all":
        frequencies = FREQUENCIES
    else:
        frequencies = [frequency]

    for year in years:
        for region in regions:
            for frequency in frequencies:
                collection, item = general(frequency, region)
                outfile = Path(__file__).parent/f"{frequency}{region}collection.json"
                outfile.parent.mkdir(exist_ok=True, parents=True)
                with open(outfile, "w") as f:
                    json.dump(collection, f, indent=2)
                outfile = Path(__file__).parent / f"{frequency}/{region}/{year}/item.json"
                with open(outfile, "w") as f:
                    json.dump(item, f, indent=2)

    if __name__ == "__main__":
        sys.exit(main())





In [27]:
fs = fsspec.filesystem('https') #, host=host)
url = f"https://noaadata.apps.nsidc.org/NOAA/G02202_V4/north/monthly"
files = [f for f in fs.find(url) if f.endswith('.nc')]
# print(files)
print(len(files))
with fs.open(files[0]) as f:
    ds = xr.open_dataset(f)

542


ValueError: found the following matches with the input file in xarray's IO backends: ['h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [24]:
cr, ir = generate('monthly', 'north', '1999')

ValueError: found the following matches with the input file in xarray's IO backends: ['h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html